<a href="https://colab.research.google.com/github/Nidarshan619/Planets/blob/master/Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install tslearn

In [35]:
import pandas as pd
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.linear_model import LogisticRegression
from tslearn.utils import to_time_series
from tslearn.utils import to_time_series_dataset
from tslearn.clustering import TimeSeriesKMeans
from tslearn.barycenters import dtw_barycenter_averaging
from tslearn.metrics import dtw, dtw_path
from tslearn.clustering import silhouette_score

In [25]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [36]:
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/Shreya_PMU/Shreya_R1.csv')

In [37]:
#df = pd.read_csv("Shreya_R1.csv")
df.head()

,Va_mag,Vb_mag,Vc_mag,Ia_mag,Ib_mag,Ic_mag,V1_mag,I1_mag,V2_mag,V0_mag,I0_mag,I2_mag,Va_ang,Vb_ang,Vc_ang,Ia_ang,Ib_ang,Ic_ang,V1_ang,I1_ang,V2_ang,V0_ang,I0_ang,I2_ang,Freq,ROCOF,Events
0,10646.684570,10646.748047,10646.700195,491.170166,491.169495,491.176086,10646.710938,491.171936,0.038324,0.000560,0.000224,0.004219,-121.217033,118.782875,-1.217377,49.385487,-70.613655,169.385956,-121.217186,49.385933,12.307068,-72.562317,47.214073,-66.098442,59.998001,0.0,0
1,10646.711914,10646.721680,10646.696289,491.171570,491.168671,491.175568,10646.709961,491.171936,0.015268,0.000735,0.000235,0.004016,-121.227867,118.771988,-1.227984,49.374691,-70.624550,169.375351,-121.227936,49.375156,-38.580269,-63.057339,43.299839,-47.018227,59.998001,0.0,0
2,10646.691406,10646.720703,10646.729492,491.170837,491.168182,491.177185,10646.713867,491.172058,0.023471,0.000662,0.000224,0.005358,-121.238754,118.761391,-1.238818,49.363773,-70.635170,169.364548,-121.238724,49.364391,70.967033,-64.459114,43.753063,-54.615639,59.998001,0.0,0
3,10646.684570,10646.747070,10646.700195,491.170135,491.169464,491.176025,10646.710938,491.171906,0.037998,0.000697,0.000234,0.004217,-121.249352,118.750557,-1.249694,49.353172,-70.645973,169.353638,-121.249489,49.353615,12.857522,-71.361115,45.862991,-66.316330,59.998001,0.0,0
4,10646.709961,10646.719727,10646.694336,491.171478,491.168579,491.175507,10646.708008,491.171844,0.014520,0.000588,0.000219,0.004026,-121.260185,118.739685,-1.260298,49.342381,-70.656876,169.343018,-121.260269,49.342842,-38.431492,-51.785282,42.666668,-47.099350,59.998001,0.0,0


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2184852 entries, 0 to 2184851
Data columns (total 27 columns):
 #   Column  Dtype  
---  ------  -----  
 0   Va_mag  float64
 1   Vb_mag  float64
 2   Vc_mag  float64
 3   Ia_mag  float64
 4   Ib_mag  float64
 5   Ic_mag  float64
 6   V1_mag  float64
 7   I1_mag  float64
 8   V2_mag  float64
 9   V0_mag  float64
 10  I0_mag  float64
 11  I2_mag  float64
 12  Va_ang  float64
 13  Vb_ang  float64
 14  Vc_ang  float64
 15  Ia_ang  float64
 16  Ib_ang  float64
 17  Ic_ang  float64
 18  V1_ang  float64
 19  I1_ang  float64
 20  V2_ang  float64
 21  V0_ang  float64
 22  I0_ang  float64
 23  I2_ang  float64
 24  Freq    float64
 25  ROCOF   float64
 26  Events  int64  
dtypes: float64(26), int64(1)
memory usage: 450.1 MB


In [39]:
df["I1_ang"] = df.I1_ang-df.V1_ang
df["V2_ang"] = df.V2_ang-df.V1_ang
df["V0_ang"] = df.V0_ang-df.V1_ang
df["I0_ang"] = df.I0_ang-df.V1_ang
df["I2_ang"] = df.I2_ang-df.V1_ang
df["V_m"] = (df["Va_mag"]+df["Vb_mag"]+df["Vc_mag"])/3
df = df.drop(["V1_ang","Va_ang","Vb_ang","Vc_ang","Ia_ang","Ib_ang","Ic_ang"],axis=1)
df.shape

(2184852, 21)

In [40]:
#To generate a book-keeping scheme. 
def diffe(df):
    df["diff"] = df.Events.diff()
    df.loc[pd.isnull(df["diff"]),"diff"] = 0   # This is just to remove nan generated from the previous step
    df["Indexing"] = np.array(range(1,df.shape[0]+1))  # Just created a false indexing starting from 1 and running till the end
    indices = list(df.loc[df["diff"]!=0,"Indexing"])  #create a list [] containing location of diffs that are not equal to 0 
    indices.insert(0,1) # included 1 at the fist change
    indices.insert(len(indices),df.shape[0])
    return indices,df

# THis function is returning a list of indices where a new event is starting 

In [22]:
df = df.loc[df.Events!=0]
df_lr = df
#df_lr = df.loc[ (df.Events == 8) |(df.Events == 1)]
indices,df_lr = diffe(df_lr)  #Function call
df_lr.shape
#df_va = df_lr[['Va_mag','Events','diff','Indexing']]


(173463, 23)

Decoding the first enigma box

In [ ]:
df["diff"] = df['Events'].diff()
df["diff"].unique()
df.loc[pd.isnull(df["diff"]),"diff"] = 0 # loc has rows 
df["Indexing"] = np.array(range(1,df.shape[0]+1))
type(df["Indexing"])
df["Indexing"]  
df['diff'].value_counts() 
indices = list(df.loc[df['diff']!=0,"Indexing"])   # Place where there is a new calss updated
indices.insert(0,1)
indices[0:5]
len(indices)
indices[3410]
indices.insert(len(indices),df.shape[0])
indices[0:10]
df.loc[1175:1179,'Events']

In [33]:
df['Events'].loc[90:93]
indices[0:10]


[1, 92, 182, 273, 363, 453, 812, 902, 992, 1112]

In [15]:
lower = [0,0,0,0,0,0,0,0,0,0]
upper = [88,88,88,88,88,88,88,88,88,88]
def transposed(df, df_va,indices):
    data_df = pd.DataFrame()
    temp_event = []
    for i in range(1,len(indices)):
        temp = df.loc[(df.Indexing>=indices[i-1])&(df.Indexing<indices[i])]  # And opertation True * True = Ture 
        t_events = temp.Events.unique()
        temp = temp.drop(["Events","diff","Indexing"],1).iloc[lower[int(temp.Events.unique())]:upper[int(temp.Events.unique())],:]  # Too much complicated for simple things
        list_temp = []
        temp_event.extend(t_events)
        for col in temp.columns:
            list_temp.extend(list(temp[col]))
        data_df[str(i)] = list_temp
    data_df = pd.DataFrame(np.transpose(np.array(data_df)))
    data_df["Event_Label"] = np.array(temp_event)
    return data_df

In [18]:
data_df = transposed(df_lr, df_va,indices) # Function called
data_df

data_df['Event_Label'].unique()

array([9, 5, 2, 1, 7, 4, 3, 8, 6])

In [19]:
data_df.shape

(1550, 1761)